# Querying models

In [1]:
import requests
import json
import subprocess
import base64

In [5]:
# Query stress detection model
url = "https://stress-detection-ai-edge-project.apps.sno.pemlab.rdu2.redhat.com/v2/models/stress-detection/infer"

#token = subprocess.check_output(["oc", "get", "secret", "default-name-stress-detection-sa", "-n", "ai-edge-project", "-o", "jsonpath={.data.token}"], text=True).strip()
#token = base64.b64decode(token).decode

token = "eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJhaS1lZGdlLXByb2plY3QiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlY3JldC5uYW1lIjoiZGVmYXVsdC1uYW1lLXN0cmVzcy1kZXRlY3Rpb24tc2EiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlcnZpY2UtYWNjb3VudC5uYW1lIjoic3RyZXNzLWRldGVjdGlvbi1zYSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50LnVpZCI6IjcxODg5Yzk3LWVjNzMtNGJlYS04MTU3LWZmM2VkMmM0N2U5MCIsInN1YiI6InN5c3RlbTpzZXJ2aWNlYWNjb3VudDphaS1lZGdlLXByb2plY3Q6c3RyZXNzLWRldGVjdGlvbi1zYSJ9.PLiSp0FEuh0Y5L1nmocMgVZlzsnjjDhFIRSbwGrJRZU1Sd7AIjnwpUr8AzOAxp1m4ladB53hR9Rqy2H0qgY3-Bg673t871icWEoHZehmulG9lU5b0zNnpkPXV52qvPucbNPQv4wFPzcFUGFn0ryd4rSVjTuu--uU9636nzfxJWqfEOaXACr5MwWS9trJnkkCJz4PeaYhMDt4JFv61nss4slPMGGQVh9VYVry5Pg-sa-wy6xXpmPXuTtE6KxBmBrGS2wm_JI4yHp1kDKw_ZgzYMvcOuf0S0-W1vpsIM4aORom45uKbBVW38iROsYaW5O3D18O4AWl2oF3eAtbsMQ-jzwUUUaIxc5nwdq8bFWlxn7uXiyxLjtPFrHOxlJ_fTeEZMscC3H9QWr5RQEf_6hJW14lzWOZKd0u_sxbdQzwYtwivF7A8pw3HTawlgSwc75vZ-rYl0D2i9s6DvF9Xpl4oO9tWac8XEZU1nTg_JpPqiH_Qx2_zxI--NTHmVBnf-dJxYJXBd_FUjV_Mq0M8O4dxLBlKggTIonvYFNQa3fqSloeLYA8TZJHnQ3UMIvqaZqzk88Bk7dnCJgXFf39V_Nccy6WunSMhYfixmjgt9BjZF5wY8xezd-dYjVPUZUXbefj43wU9aYKtmwkmn8dswNQl0Rluxc6sJ_-hs3HTyey80k"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

data = {
    "inputs": [
        {
            "name": "inputs",
            "shape": [1, 9], 
            "datatype": "FP32",
            "data": [[0.15, 97.00, 480.00, 355.00, 150.00, 50.00, 55.00, 22.00, 100.00]]
        }
    ]
}

response = requests.post(url, headers=headers, json=data, verify=True)


def process_response(response):
    if response.status_code == 200:
        result = response.json()["outputs"][0]["data"][-1]
        print("Stress Prediction:", "STRESSED" if result == 1.0 else "NORMAL")
    else:
        print(f"Error: {response.status_code}, {response.text}")


process_response(response)

Stress Prediction: STRESSED


In [6]:
# Query time to failure model
url = "https://time-to-failure-ai-edge-project.apps.sno.pemlab.rdu2.redhat.com/v2/models/time-to-failure/infer"

token = "eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJhaS1lZGdlLXByb2plY3QiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlY3JldC5uYW1lIjoiZGVmYXVsdC1uYW1lLXRpbWUtdG8tZmFpbHVyZS1zYSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50Lm5hbWUiOiJ0aW1lLXRvLWZhaWx1cmUtc2EiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlcnZpY2UtYWNjb3VudC51aWQiOiJlNTE5YzljYy0wYWI0LTQ5MzctOTcyYi1iYzQ3YTAzYmU0NTIiLCJzdWIiOiJzeXN0ZW06c2VydmljZWFjY291bnQ6YWktZWRnZS1wcm9qZWN0OnRpbWUtdG8tZmFpbHVyZS1zYSJ9.N2rb2XJpG4ieE25jmuw9SIIyCh4CAbDhhEV6HsNtkV0EhHNrhcS4KLbbsI8BK8pXBpd7Pr6E7kYCU_5SKDguLQvjFVZqKYY_c2f5RKbmQ_79t9w5PfPBiquQBQEp60aKWiApJQPgnLSFdxMbf-XYwaMfhM-VWy5lLEHEEVGWURfu1HiY1dmQOo3oVnE6olkCnq3lyXhen0Ym3L8zPIy3-KweaB80VIqMWhzIpbLUISTygVU3nEUoyliw9PJgTWUTyhIs0BSDwmSbmA0nc4kuToLODBsGVVlt8xA9a-TgdX8GkTB9f-Z-Z_htJRgutJHKXYpDBk51zMU5gGU6ym6ZXCNowFQdQkTnc79h4Y13dvplgUDoqcv4iQLybsO8YK99yp5VtMQfCejP0RG3uhjh52GsFukk_kC9bSW0Ah7RCJgh6ujPUQsFCU6imnPwmoeAkZGxOf_RP6wBbDfTtg9uBw7cYayEUjPv8KYKpvJPhYPuXX77qjqVR_asqz0vmESOZAKKeLhJmw94373xATHPpw42Vp8lAM4sLttW0UkKseHy-8tiHgvecGC5eMmA3NnSoeTQ6jMYQ985gVElDfvDerb1XHtNGbVqvXwhd8LXWuKSaVB_RIU6Ifi3cn7ruwHvJGXB8QwHunoKYR2lme5DAglXAHGbY-QaMbI20CILNFU"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

data = {
    "inputs": [
        {
            "name": "inputs",
            "shape": [1, 5], 
            "datatype": "FP32",
            "data": [[45.0, 500.0, 370.0, 0.4, 98.0]]
        }
    ]
}

response = requests.post(url, headers=headers, json=data, verify=True)


def process_response(response):
    if response.status_code == 200:
        result = response.json()
        output_value = result["outputs"][0]["data"][0]  
        print(f"Predicted Time Before Failure: {output_value:.2f} hours")
    else:
        print(f"Error: {response.status_code}, {response.text}")


process_response(response)

Predicted Time Before Failure: 64.57 hours
